# 1. 교차로 위도, 경도 정보 가져오기

참고 : https://github.com/JunPyoPark/Networkx-Projects/tree/master/울산%20도로교통정보  
( https://junpyopark.github.io/road-network-construction-1/)

###1. google drive mount 및 필요 라이브러리 가져오기

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

drive = './gdrive/My Drive/Colab Notebooks/Lumos_9X/Test'
!ls -al './gdrive/My Drive/Colab Notebooks/Lumos_9X/Test'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
total 786
drwx------ 2 root root   4096 May  4 07:45 data
-rw------- 1 root root 183917 May  4 10:04 Incheon_links.csv
-rw------- 1 root root  53145 May  4 10:29 Incheon_nodes2.csv
-rw------- 1 root root  48965 May  4 10:04 Incheon_nodes.csv
-rw------- 1 root root  10909 May  5 06:11 intersection_1.ipynb
-rw------- 1 root root 187951 May  4 10:29 link2.csv
-rw------- 1 root root 187951 May  4 16:35 link.csv
-rw------- 1 root root  53145 May  4 10:29 nodes2.cs

In [4]:
#!pip install pyshp
#!pip install pyproj

     |████████████████████████████████| 10.8MB 2.8MB/s 


In [0]:

import shapefile #pyshp module.
import pandas as pd
import numpy as np
from pyproj import Proj, transform #pyproj module.
import networkx as nx

###2. shp file 이용해서 dataframe에 저장하기 

In [0]:
#file 읽기 (encoding 주의)
shp_path_node = './gdrive/My Drive/Colab Notebooks/Lumos_9X/Test/data/NodeLink/MOCT_NODE.shp'
sf_node = shapefile.Reader(shp_path_node, encoding = 'cp949')
shp_path_link = './gdrive/My Drive/Colab Notebooks/Lumos_9X/Test/data/NodeLink/MOCT_LINK.shp'
sf_link = shapefile.Reader(shp_path_link, encoding = 'cp949')


In [7]:

#1. 필드명 두번째 줄부터 뽑아서 리스트 형태로 저장하기
fields_node = [x[0] for x in sf_node.fields][1:]
fields_link = [x[0] for x in sf_link.fields][1:]
#len(fields_node) #11


#2. 데이터 
records_node = sf_node.records()
#len(records_node) #120630
records_link = sf_link.records()

#3 노드의 좌표 데이터 저장
shps = [s.points for s in sf_node.shapes()]
shps[1:3]

# [
# [[229535.21800000034, 250194.52040000074]],
# [[229465.27020000014, 249416.52300000004]]
# ]



[[[229535.21800000034, 250194.52040000074]],
 [[229465.27020000014, 249416.52300000004]]]

In [8]:
#4. dataframe으로 저장
##4_1. 노드 데이터
node_dataframe = pd.DataFrame(columns = fields_node, data = records_node )
###coords 열 생성해서 좌표 데이터 추가하기
node_dataframe = node_dataframe.assign(coords=shps)

##4_2. 링크 데이터
link_dataframe = pd.DataFrame(columns=fields_link, data=records_link)

node_dataframe.head(5)

,NODE_ID,NODE_TYPE,NODE_NAME,TURN_P,REMARK,USER_ID,WORKSTATE,DEPT_CODE,STNL_REG,TMPID,UPLOAD_ID,coords
0,3330017400,101,고읍교차로,0,,korea,5,,333,OEXT3330017400,,"[[229757.35709999967, 251792.7910999991]]"
1,3330016500,104,벌교터널,0,,korea,5,,333,OEXT3330016500,,"[[229535.21800000034, 250194.52040000074]]"
2,3330015600,104,벌교터널,0,,korea,5,,333,OEXT3330015600,,"[[229465.27020000014, 249416.52300000004]]"
3,3330014900,101,벌교교차로,0,,korea,5,,333,OEXT3330014900,,"[[229455.64099999983, 249249.00129999965]]"
4,3330014600,101,벌교교차로,0,,korea,5,,333,OEXT3330014600,,"[[229412.60429999977, 249169.92640000023]]"


###3. 원하는 지역의 정보만 가져오기

Lumos test에서는 강남구 양재동 일대를 대상으로 한다.   
행자부코드 : 11680  (강남구)
권역코드     : 122 (서울특별시)

http://nodelink.its.go.kr/intro/intro06_05.aspx  << 에서 확인

In [9]:
df_node = node_dataframe[node_dataframe['STNL_REG'] == str(122)]
df_link = link_dataframe[link_dataframe['STNL_REG'] == str(122)]

print(df_node.shape, df_link.shape)
df_node.tail(10)


(507, 12) (1462, 25)


,NODE_ID,NODE_TYPE,NODE_NAME,TURN_P,REMARK,USER_ID,WORKSTATE,DEPT_CODE,STNL_REG,TMPID,UPLOAD_ID,coords
98510,1220063800,102,현대3차아파트63동,0,,,5,11680,122,TMP201410N10026,,"[[202968.01609999966, 548120.1201000009]]"
98511,1220063900,102,현대3차아파트95동,0,,,5,11680,122,TMP201410N10025,,"[[203099.25600000005, 548082.2596000005]]"
98528,1220064000,108,영동대로_올림픽대로_진,0,,,5,11680,122,TMP201410N10008,,"[[204894.0876000002, 547091.4360000007]]"
98534,1220064100,108,올림픽대로_동부간선도,0,,,5,11680,122,TMP201410N10002,,"[[205886.9548000004, 546338.5817000009]]"
99630,1220116500,104,위례터널(서측),0,,mappers,5,60190,122,TMPID20160720552,mappers20160812105536,"[[210091.03019999992, 540679.9328000005]]"
106491,1220014600,101,숯내교(서단),0,,mappers,5,60190,122,,mappers20170823151004,"[[210103.54849999957, 542105.6202000007]]"
106799,1220147200,101,서울율현초교,0,,mappers,5,60190,122,TMPID20170720437,mappers20170823151004,"[[209454.27550000045, 541548.4072999991]]"
106811,1220148400,101,서울세곡2지구,0,,mappers,5,60190,122,TMPID20170720436,mappers20170823151004,"[[210337.71530000027, 541685.0775000006]]"
106812,1220148500,101,율현2빗물펌프장,0,,mappers,5,60190,122,TMPID20170720435,mappers20170823151004,"[[210544.89419999998, 541317.3352000006]]"
120495,1220150600,104,구룡지하차도(동측),0,,nodelinkmanager,5,50000,122,TMPID20190419154217,nodelinkmanager2019042413,"[[204867.67470000032, 541622.5287999995]]"


###4. korea2000 좌표계 -> wgs84 (위/경도) 변환


In [11]:
# korea 2000/central belt 2010 (epsg:5186) to wgs84(epsg:4326)
inProj = Proj(init = 'epsg:5186') #korea2000
outProj= Proj(init = 'epsg:4326') #wgs84

#del df_node['latitude'] # delete coords
#del df_node['longitude'] # delete coords

new_coords = df_node['coords'].apply( lambda x : transform(inProj,outProj,x[0][0],x[0][1]))
df_node['latitude'] = new_coords.apply(lambda x : x[1])
df_node['longitude'] = new_coords.apply(lambda x : x[0])



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [12]:
#del df_node['coords']
df_node.head(5)


,NODE_ID,NODE_TYPE,NODE_NAME,TURN_P,REMARK,USER_ID,WORKSTATE,DEPT_CODE,STNL_REG,TMPID,UPLOAD_ID,coords,latitude,longitude
91233,1220001100,108,동부간선도로-남부순환,0,,,5,11680,122,,,"[[208007.12170000002, 544078.7682000007]]",37.496133,127.090548
91234,1220000700,108,수서IC2,0,,,5,11680,122,,,"[[209164.89719999954, 543524.8106999993]]",37.491131,127.103633
91235,1220001500,108,동부간선도로-남부순환,0,,,5,11680,122,,,"[[206844.69820000045, 544568.4737]]",37.500554,127.077407
91236,1220025600,108,수서IC6,0,,,5,11680,122,,,"[[208477.72869999986, 543632.6754000001]]",37.492109,127.095864
91237,1220058500,108,올림픽대로-일반국도47,0,,,5,11680,122,,,"[[204564.9890999999, 547683.1651000008]]",37.528632,127.051645


###5. 셀 이름 변경

In [13]:
# Change column name to draw network in Gephi
df_node.rename(columns={'NODE_ID':'Id'},inplace = True)
df_link.rename(columns={'F_NODE':'Source','T_NODE':'Target'},inplace = True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [14]:
df_link.head(5)

,LINK_ID,Source,Target,ROAD_USE,LANES,ROAD_RANK,ROAD_NAME,MULTI_LINK,CONNECT,MAX_SPD,...,WORKSTATE,DEPT_CODE,STNL_REG,ROAD_TYPE,ROAD_NO,TMPID,UPLOAD_ID,SOSFNODEID,SOSTNODEID,SHAPE_STLe
126790,1220110900,1220063900,1220060300,0,1.0,107,선릉로189길,0,000,30.0,...,5,11680,122,000,-,TMP201410L10122,,TMP201410N10025,,126.845777
126791,1220111000,1220060300,1220063900,0,1.0,107,선릉로189길,0,000,30.0,...,5,11680,122,000,-,TMP201410L10123,,,TMP201410N10025,126.845777
126792,1220109700,1220035000,1220035500,1,1.0,107,선릉로89길,0,000,60.0,...,5,11680,122,000,-,TMP201410L10124,,,,168.723197
126793,1220110500,1220045700,1220046500,1,1.0,107,봉은사로113길,0,000,40.0,...,5,11680,122,000,-,TMP201410L10125,,,,68.769505
126794,1220110300,1220044800,1220045700,1,2.0,107,봉은사로113길,0,000,40.0,...,5,11680,122,000,-,TMP201410L10126,,,,78.915399


###6. csv로 저장

In [0]:
df_node.to_csv('./gdrive/My Drive/Colab Notebooks/Lumos_9X/Test/nodes.csv', encoding = 'utf-8' ) # node list
df_link.to_csv('./gdrive/My Drive/Colab Notebooks/Lumos_9X/Test/link.csv', encoding = 'utf-8') # edge(=link) list

#2. 도로 네트워크 시각화
###1. 라이브러리 import 및 파일 가져오기


In [16]:
import os
import folium
import networkx as nx

nodes = pd.read_csv('./gdrive/My Drive/Colab Notebooks/Lumos_9X/Test/nodes.csv')
links = pd.read_csv('./gdrive/My Drive/Colab Notebooks/Lumos_9X/Test/link.csv')


#필요한 열만 가져오기
nodes = nodes[['Id','NODE_NAME','latitude','longitude']]
links = links[['Source','Target']]

nodes.head(3)

,Id,NODE_NAME,latitude,longitude
0,1220001100,동부간선도로-남부순환,37.496133,127.090548
1,1220000700,수서IC2,37.491131,127.103633
2,1220001500,동부간선도로-남부순환,37.500554,127.077407


In [17]:
links.head(3)

,Source,Target
0,1220063900,1220060300
1,1220060300,1220063900
2,1220035000,1220035500


### 2. 필요 지역 이외의 노드와 연결된 데이터 제거

In [18]:
source_in = links['Source'].apply(lambda x : x in list(nodes['Id'])) # check Sources are in incheon_id #시작점
target_in = links['Target'].apply(lambda x : x in list(nodes['Id'])) # check Targets are in incheon_id #끝점


# source_in and target_in are boolean type pandas.Series which contains True or False
gangnam_links = links[source_in & target_in] # contain if both target and source are contained in incheon_id

print("전체 노드수: " , len(links))
print("강남 노드수: " , len(gangnam_links))

전체 노드수:  1462
강남 노드수:  1358


###3. nx.Graph에 노드, 링크 추가

In [19]:
G = nx.Graph()
# R is the Earth's radius
R = 6371e3

#그래프에 노드정보 추가
for idx,row in nodes.iterrows():
    # add node to Graph G
    G.add_node(row['Id'],Label=row['NODE_NAME'],latitude=row['latitude'], longitude=row['longitude'])

    
#그래프에 링크추가 (가중치)
for idx,row in gangnam_links.iterrows():
    ## Calculate the distance between Source and Target Nodes
    lon1 = float(nodes[nodes['Id'] == row['Source']]['longitude'] * np.pi/180)
    lat1 = float(nodes[nodes['Id'] == row['Source']]['latitude'] * np.pi/180)
    lon2 = float(nodes[nodes['Id'] == row['Target']]['longitude'] * np.pi/180)
    lat2 = float(nodes[nodes['Id'] == row['Target']]['latitude'] * np.pi/180)
    
    d_lat = lat2 - lat1
    d_lon = lon2 - lon1
    a = np.sin(d_lat/2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(d_lon/2) ** 2
    c = 2 * np.arctan2(a**0.5, (1-a) ** 0.5)
    d = R * c #가중치 
    
    # Link attribute : 'Source', 'Target' and weight = 'Length between them'
    G.add_edge(row['Source'],row['Target'],weight = d)
    

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 507
Number of edges: 795
Average degree:   3.1361


### Folium 을 통한 시각화

In [21]:
# Positioning the Standard Point for our Folium Map
std_point = tuple(nodes.head(1)[['latitude','longitude']].iloc[0]) #첫번째노드
std_point

(37.496132820916976, 127.09054755678122)

In [22]:
map_osm = folium.Map(location=std_point, zoom_start=17) 
# location : 기준이 되는 점, zoom_start : 지도 상의 zoom level 을 나타냅니다.

for ix, row in nodes.iterrows():
    location = (row['latitude'], row['longitude']) # 위도, 경도 튜플
    
    folium.Circle(
        location=location,
        radius=G.degree[row['Id']] * 10, # 지름이 degree에 비례하도록 설정
        color='white',
        weight=1,
        fill_opacity=0.6,
        opacity=1,
        fill_color='red',
        fill=True,  # gets overridden by fill_color
        # popup=str(row['Id'])
    ).add_to(map_osm)
    # folium.Marker(location, popup=row['NODE_NAME']).add_to(map_osm)
    
    
kw = {'opacity': 0.5, 'weight': 2}
for ix, row in gangnam_links.iterrows():
    start = tuple(nodes[nodes['Id']==row['Source']][['latitude','longitude']].iloc[0])
    end = tuple(nodes[nodes['Id']==row['Target']][['latitude','longitude']].iloc[0])
    folium.PolyLine(
        locations=[start, end],
        color='blue',
        line_cap='round',
        **kw,
    ).add_to(map_osm)
# it takes some time.....

map_osm